In [ ]:
# default_exp utils

# utils

> General helper functions

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import functools
import tempfile
import pkgutil
from configparser import ConfigParser

In [ ]:
#export
def check_git_dir():
    "Check whether there is a .git directory in your current folder"
    return bool(os.path.exists(".git"))

In [ ]:
#export
def check_project_root(func):
    """Check that the user is on the project root directory by checking if there is a .git folder
    before running `func`
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if not check_git_dir:
            raise Exception("You need to be on your project root to execute this command")
        else:
            return func(*args, **kwargs)
    return wrapper

In [ ]:
#export
@check_project_root
def read_credentials(path):
    "Read credentials from `path`"
    config = ConfigParser(delimiters=['='])
    with open(path, "r") as f:
        config.read_string("[DEFAULT]\n" + f.read())
    return config["DEFAULT"]

In [ ]:
if not check_git_dir:
    os.chdir("..")  # we need this if we test locally in the notebook itself
input_credentials = "user = SomeUser\npassword = SomePassword"
expected_credentials = {"user": "SomeUser", "password": "SomePassword"}
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_file = os.path.join(tmp_dir, "tmp")
    with open(tmp_file, "w") as f:
        f.write(input_credentials)
    credentials = read_credentials(tmp_file)
assert credentials == expected_credentials

In [ ]:
#export
def get_config():
    "Get config from settings.ini"
    config = ConfigParser(delimiters=['='])
    config.read('settings.ini')
    cfg = config['DEFAULT']
    return cfg

In [ ]:
#export
def get_template(name):
    "Get template with filename `name`"
    template = pkgutil.get_data(__name__, f"templates/{name}").decode("utf-8")
    return template

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_gh.ipynb.
Converted 04_utils.ipynb.
Converted index.ipynb.
